In [17]:
import os
import random
from time import sleep
import bioblend
from bioblend.galaxy import GalaxyInstance
from bioblend.galaxy import histories


server = 'http://127.0.0.1:9090/'
key = '7a9d91ab284dae8207afceab171c76d4'
ipy_file_path = "scikit-learn.ipynb"


def run_dynamic_code(server, key, ipy_file_path, tool_name="run_jupyter_job"):
    # create connection
    gi = GalaxyInstance(server, key=key)
    history = histories.HistoryClient(gi)
    new_history = history.create_history(str(random.randint(1, 10000000)))
    # get script
    from nbformat import read, NO_CONVERT
    with open(ipy_file_path) as fp:
        notebook = read(fp, NO_CONVERT)
    cells = notebook['cells']
    code_cells = [c for c in cells if c['cell_type'] == 'code']
    notebook_script = ""
    for cell in code_cells:
        notebook_script += cell.source + "\n\n"
    target_file_name = "target-file.py"
    with open(target_file_name, "w") as f_obj:
        f_obj.write(notebook_script)
    # upload script
    uploaded_dataset = gi.tools.upload_file(target_file_name, new_history["id"])
    sleep(20)
    hist_id = new_history["id"]
    uploaded_file_path = uploaded_dataset["outputs"][0]["id"]
    tool_run = gi.tools.run_tool(hist_id, tool_name, {"inputs": {"select_file": uploaded_file_path}})
    print(tool_run["jobs"][0]["state"])

run_dynamic_code(server, key, ipy_file_path)

new
